## Esse notebook tem o objetivo de documentar a evolução do projeto

In [5]:
import pyspark
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.sql import SQLContext
import json
from pyspark.sql.functions import mean, min, max

from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MinhaAplicacao").getOrCreate()
sc = spark.sparkContext
sqlContext = pyspark.SQLContext(sc)

df= spark.read.json('../yelp_academic_dataset_review.json')
df.show()

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|ujmEBvifdJM6h6RLv...|   0|2013-05-07 04:34:36|    1|Q1sbwvVQXV2734tPg...|  1.0|Total bill for th...|     6|hG7b0MtEbXx5QzbzE...|
|NZnhc2sEQy3RmzKTZ...|   0|2017-01-14 21:30:33|    0|GJXCdrto3ASJOqKeV...|  5.0|I *adore* Travis ...|     0|yXQM5uF2jS6es16SJ...|
|WTqjgwHlXbSFevF32...|   0|2016-11-09 20:09:03|    0|2TzJjDVDEuAW6MR5V...|  5.0|I have to say tha...|     3|n6-Gk65cPZL6Uz8qR...|
|ikCg8xy5JIg_NGPx-...|   0|2018-01-09 20:56:38|    0|yi0R0Ugj_xUx_Nek0...|  5.0|Went in for a lun...|     0|dacAIZ6fTM6mqwW5u...|
|b1b1eb3uo-w561D0Z...|   0|2018-01-30 23:07:38|    0|11a8sVPMUFtaC7_AB...|  1.0|Today was 

In [6]:
print((df.count(), len(df.columns)))

(6685900, 9)


In [7]:
df.first()

Row(business_id='ujmEBvifdJM6h6RLv4wQIg', cool=0, date='2013-05-07 04:34:36', funny=1, review_id='Q1sbwvVQXV2734tPgoKj4Q', stars=1.0, text='Total bill for this horrible service? Over $8Gs. These crooks actually had the nerve to charge us $69 for 3 pills. I checked online the pills can be had for 19 cents EACH! Avoid Hospital ERs at all costs.', useful=6, user_id='hG7b0MtEbXx5QzbzE6C_VA')

In [8]:
df.describe().show()

+-------+--------------------+------------------+-------------------+-------------------+--------------------+------------------+------------------------------------+------------------+--------------------+
|summary|         business_id|              cool|               date|              funny|           review_id|             stars|                                text|            useful|             user_id|
+-------+--------------------+------------------+-------------------+-------------------+--------------------+------------------+------------------------------------+------------------+--------------------+
|  count|             6685900|           6685900|            6685900|            6685900|             6685900|           6685900|                             6685900|           6685900|             6685900|
|   mean|                null|0.5787708461089756|               null|0.48276671801851656|                null|3.7161993149762935|                                null|1.3541

In [14]:
gr = df.agg(countDistinct("user_id"))
gr.show()

+-----------------------+
|count(DISTINCT user_id)|
+-----------------------+
|                1637138|
+-----------------------+



## Explorando as possibilidades com um business especifico

In [10]:
def get_business(biz_id):   
    df_b = df.filter(df['business_id'] == biz_id).select('stars','date','text','useful') # Separa o df com somente o business de intresse

    df_size = df_b.count() # Conta o numero de reviews
    avg_rating = (df_b.agg({"stars": "avg"})).first()[0] # Faz a media de reviews
    std = (df_b.agg({"stars": "stddev"})).first()[0] # Calculda o desvio padrao
    worst = df_b.orderBy(asc("stars"),desc("useful")).limit(1).first() # Seleciona a review com nota mais baixa e mais marcada como 'useful'
    
    print("Quantidade de Reviews: ", df_size)
    print("Media das Avaliacoes: ", avg_rating)
    print("Desvio padrao: ", std)
    print("\nPior review: ", worst['stars'], "estrelas em ", worst['date'])
    print("Comentario da pior review: ", worst['text'])

In [11]:
#biz_id = 'Gyrez6K8f1AyR7dzW9fvAw'
biz_id = 'FKQUxOfW_W5R4OTut4G1Vw'
#NZnhc2sEQy3RmzKTZnqtwQ

get_business(biz_id)

Quantidade de Reviews:  39
Media das Avaliacoes:  4.230769230769231
Desvio padrao:  0.9586596581065523

Pior review:  1.0 estrelas em  2011-02-19 20:35:31
Comentario da pior review:  I found a short black curly hair in my basket of fries... and it was definitely not mine.

Needless to say, this completely ruined my meal. Thanks to Peppi's, I won't be eating anything that isn't ready-to-eat as is (like fruit) for the next few days.

Not only does the shop LOOK grimey, it IS grimey... Completely disgusted.


## Pegando informacoes sobre o business na sua pagina no Yelp

In [61]:
from bs4 import BeautifulSoup
import requests
import re

def url_to_string(url):
    res = requests.get(url)
    html = res.text
    soup = BeautifulSoup(html, 'html5lib')
    for script in soup(["script", "style", 'aside']):
        script.extract()
    return " ".join(re.split(r'[\n\t]+', soup.get_text()))

In [62]:
ny_bb = url_to_string('https://www.yelp.com/biz/'+biz_id)
# article = nlp(ny_bb)
# len(article.ents)

In [63]:
#ny_bb

In [81]:
res = requests.get('https://www.yelp.com/biz_attribute?biz_id=FKQUxOfW_W5R4OTut4G1Vw')#'https://www.yelp.com/biz/'+biz_id)
html =res.text
soup = BeautifulSoup(html, 'html5lib')


In [82]:
soup

<!DOCTYPE html>
<!--[if lt IE 7 ]> <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie6 ie ltie9 ltie8 no-js" lang="en"> <![endif]--><!--[if IE 7 ]>    <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie7 ie ltie9 ltie8 no-js" lang="en"> <![endif]--><!--[if IE 8 ]>    <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie8 ie ltie9 no-js" lang="en"> <![endif]--><!--[if IE 9 ]>    <html xmlns:fb="http://www.facebook.com/2008/fbml" class="ie9 ie no-js" lang="en"> <![endif]--><!--[if (gt IE 9)|!(IE)]><!--><html class="no-js" lang="en" xmlns:fb="http://www.facebook.com/2008/fbml"><!--<![endif]--><head>
        <script>
            (function() {
                var main = null;

                var main=function(){window.onerror=function(k,a,c,i,f){var j=(document.getElementsByTagName("html")[0].getAttribute("webdriver")==="true"||navigator.userAgent==="selenium");var h=f&&(f.name==="ServerSideRenderingError"||f.name==="CSRFallbackError");if(j&&!h){document.body.inne

In [103]:
import bs4
res.raise_for_status()
recipeSoup = bs4.BeautifulSoup(res.text, "html.parser")
type(recipeSoup)

price = soup.findAll("input",{"type":"text"}) # price of restaurant
name = price[4] 
addr = price[5] 
city = price[7] 
zipcode = price[8] 
phone = price[9] 

In [118]:
price

[<input aria-hidden="true" autocomplete="off" class="input__373c0__1POmY hidden-input__373c0__3SQQL" disabled="" placeholder=" " spellcheck="false" tabindex="-1" type="text" value=""/>,
 <input aria-autocomplete="list" autocomplete="off" class="input__373c0__1POmY original-input__373c0__lpd54" data-testid="suggest-desc-input" id="search_description" name="find_desc" placeholder="tacos, cheap dinner, Max’s" tabindex="0" type="text" value=""/>,
 <input aria-hidden="true" autocomplete="off" class="input__373c0__1POmY hidden-input__373c0__3SQQL" disabled="" placeholder=" " spellcheck="false" tabindex="-1" type="text" value=""/>,
 <input aria-autocomplete="list" autocomplete="off" class="input__373c0__1POmY original-input__373c0__lpd54" data-testid="suggest-location-input" id="search_location" placeholder="address, neighborhood, city, state or zip" tabindex="0" type="text" value="San Francisco, CA"/>,
 <input class="text" id="attr_BusinessName" name="BusinessName" placeholder="Mel’s Diner" 

In [119]:
pip install yelp

  Created wheel for yelp: filename=yelp-1.0.2-cp37-none-any.whl size=11813 sha256=4ff57d5a7822379e44d96b578d3e0050024957e791aafc7924b0c8551d1891b9
  Stored in directory: C:\Users\guipl\AppData\Local\pip\Cache\wheels\a5\1f\cb\cd71331e931f7b0e719d4319b8cc14e34faabac6946ff35aa4
Successfully built yelp
Note: you may need to restart the kernel to use updated packages.


AttributeError: 'Client' object has no attribute 'business'

In [15]:
gr = df.agg(countDistinct("business_id"))
gr.show()

+---------------------------+
|count(DISTINCT business_id)|
+---------------------------+
|                     192606|
+---------------------------+



In [43]:
df_biz = df.select("business_id").distinct()

print((df_biz.count(), len(df_biz.columns)))
df_biz.show()

(192606, 1)
+--------------------+
|         business_id|
+--------------------+
|f4mh1Y0rnvbJRfQ3j...|
|cKwg6HFaLYXl7Ar0r...|
|jcpgiXF0PCyS9hrvq...|
|R_M4P9XetEM-aLE7e...|
|DEBqmgxv2yhJ93LqG...|
|Cml4Yt5cTx64cOMan...|
|bo3SQVtErnMOqO6lk...|
|Cl-xl1vTUwHeaGgBx...|
|oIEmXWLtoh5blz-iw...|
|Op2IR4FffXZ5KXYFn...|
|yB5FMuc9Y3oyhsOmu...|
|cEqOh78v1g1RCWHyu...|
|lt8IW9Bpy9GMeKGxy...|
|uC3qwaxsOkdJzpOc0...|
|686oeWNsbc-aczplC...|
|gPuxh3HNvoVt8aWVW...|
|mA27CG2U3ytmkxIGV...|
|x6qH9HXhzuKM03jcZ...|
|74LU6K2ro5AQXKT0J...|
|TdefcbsFAj6WXHwlG...|
+--------------------+
only showing top 20 rows



In [46]:
for i in range(20):
    print(df_biz[0][i])

Column<b'business_id[0]'>
Column<b'business_id[1]'>
Column<b'business_id[2]'>
Column<b'business_id[3]'>
Column<b'business_id[4]'>
Column<b'business_id[5]'>
Column<b'business_id[6]'>
Column<b'business_id[7]'>
Column<b'business_id[8]'>
Column<b'business_id[9]'>
Column<b'business_id[10]'>
Column<b'business_id[11]'>
Column<b'business_id[12]'>
Column<b'business_id[13]'>
Column<b'business_id[14]'>
Column<b'business_id[15]'>
Column<b'business_id[16]'>
Column<b'business_id[17]'>
Column<b'business_id[18]'>
Column<b'business_id[19]'>


In [55]:
def customFunction(row):
    print(row.business_id)
    return (row.business_id)

sample2 = df_biz.rdd.map(customFunction)
sample2

PythonRDD[179] at RDD at PythonRDD.scala:53

In [ ]:
for row in df_biz.rdd.collect():
    print(row.business_id)


f4mh1Y0rnvbJRfQ3jPkqzQ
cKwg6HFaLYXl7Ar0rHojAQ
jcpgiXF0PCyS9hrvqseGEA
R_M4P9XetEM-aLE7eHdthw
DEBqmgxv2yhJ93LqGpKltg
Cml4Yt5cTx64cOManHe-Gw
bo3SQVtErnMOqO6lkxWdkw
Cl-xl1vTUwHeaGgBxzdTRA
oIEmXWLtoh5blz-iwI8MZQ
Op2IR4FffXZ5KXYFnDL2yA
yB5FMuc9Y3oyhsOmuIZSpQ
cEqOh78v1g1RCWHyuHwQng
lt8IW9Bpy9GMeKGxytefVA
uC3qwaxsOkdJzpOc0v1Mog
686oeWNsbc-aczplCFa_0g
gPuxh3HNvoVt8aWVWhgGcA
mA27CG2U3ytmkxIGVnk0jQ
x6qH9HXhzuKM03jcZmg-ZQ
74LU6K2ro5AQXKT0Jf0SPw
TdefcbsFAj6WXHwlGQHIbA
E9QTQ4DOKo1UsGNmMX8sGQ
MEoDTsA3Af6TLzB7Z_7oDw
kr5JmGtVl0pUq9-KB_91QA
6QRY4tMQdHFOe2-_CUHMdA
09p3b5BCSz2FPvgR-8xz3A
aAiD7Zsq2OeEFEOWmfQ8ug
ipFreSFhjClfNETuMjBtrA
7coCBjZNMJ48BD2taPKgHQ
9ypHY6MNp3CDDdQJ_d817A
1UwaMUnVKeWcV14qvDmBRg
7IoodTC2xwmBWvoFCP5urg
RxtaiNM8wK2U924dq3siTQ
YYztMMdcudL4xuSF29yelQ
4v6e_afy2uAiQYDq9xtywQ
FpFIAW_IEvASZBbusABbkA
zhT6PXyerupZzXL-smu-ZQ
kysZp6aKd7vnV5dnYxqJEQ
n329FYFcIKmcicKyENyadQ
WvkxYyA1fXDPDyqIVn8zBQ
_crm3cMcEe1PX4H1Uap_AQ
nkh7rzfDVuyKncuKvnUx3Q
Y2bmAJy7mHhncvHk_FXjzQ
FcRf8dciFsQ9ZUj0B7yiOg
nqBLuNgN1Vr

x0PF0MvzyRrKbF00xqlIMw
ydSiR7YyALkmmACVNtXzhg
nm4mFCYISSQizmzFhevcKw
JsFC1ATvm-7n3joFeYVqKg
XH6BGTLcQPx1YPMpU98w1w
d8U6onGZkgZZDCqidqmcnA
Tb4B6YR55UQ1Lccg3RlL6A
o-Lju_Yagb-_slCHWRAv6Q
FpDYvApkcQUTCudLcuSzQQ
DNvnu5Bs12419XP93xGkZA
zQN7xr7qhq4RYLdRYEWhvw
jaJO3SyFIrHdKVtsHufsJQ
vRkTJ8EYMjFBhnsMVv6mwg
WhvRZd09bSJI178QgWxPGw
GJNwtxDuPcBVneQRYfMETg
sM8cSO14bbrXwvB7e_JY_w
q27-Sh7VJLPI24tGxE35PQ
DvJJNyOJNz5NBZOP4UScIw
FCaluJ5MZp4CHePYXjjT7w
NzhRebtw2AIxXkVO_BrOOQ
QEGRW67e9jkiFUPOmVsh0w
83BO54hWweSbbZXj6-9qQA
1p3YpI-mspxnYZI614CyXA
VFZpI9wSaFv5HJ5tjrmkEQ
tKkni4Rix-CAO3zY8JwHDw
OX0T9dWI8b7meu-ljTo22A
5T6Binl4ayX1qJ5RD7Xu_g
C1RZyLE0d97wRfZoNvWC7g
U727bIditAsUdRSWvJ9n6g
Eis_z-uF_2gUf83UE9blog
VWn0-myNv9SkyozBr2hJ0w
SS6Ow_wWjP8CbfbKEW4ZEA
J0VhheD10-2A1iYm_5T6wQ
tvYID0arhN-shKGUrC1Wsg
zxsktAMsVBKj2PvvCV11UQ
8h4Xngi0h2S3_wY3rg8dLg
VEPGew9ggS4FNM5_zD6naA
4hWDMVtfnpyY72_5QMbthA
ElcjcVC348qqJ1kTS9i7xw
AGN5OL2t2Abcpa3K8Gwsiw
HCKnxLI4uzLnskDPh8pEUA
Qb9EsZdqS_0Xac5VRQtivQ
VFbslvHQDSaJonYdxu_jrQ
xVoPh8oCb29

0mzesmyDRzZPLIgzQm5vmA
W7_DjQY-RnlGXbG29om3ZQ
d5AJ2xFmc22qs_jLqVg0Xw
SIZOo8ErmV3k3Y_7PHeNHQ
yFFFiuPQdHr-KPqD9s2LsQ
AQ5hsa-yAr4pQ5c9XlX8kA
2s6AF8AR5H0BlJlyYTCG3A
mMxjpJReJtfYQwwEoImdZA
gncaq62JXkXqmpfb4P3tTA
H3XxfTpsToyWCgmufacZ3w
FJ6UWsmGBmEpOFWdf7UsFA
POw6bMXjMHPbV55xVfvAag
AqcNtEfFuxRn0b81t8iC2w
VRo65AO8tmtgwmnMtfvWFA
uXLxJw-dx5tT7XRaR2utog
GGbo2tJk2yQuPxO5aWG3rg
gDKumf8oAFUnIDZbAr65og
ctqV64UcFrd3q_BFjwfqXw
Z4ziFUcsdCF_5x0OytwLsg
qz1zu28a44yhPtoBqyUP7g
rg_OQ4NjMGbeJYh5zHEINQ
5mUtUODlMOItgwALZOItpQ
yOeqMJ5dyXX-ImaZr27sbQ
wNbMERhha6uLREkrLobP5A
SDQjsedYWAWMa37NC1o4tw
5J_ehn980bR2LjOeLycZyA
_lLI8zKjvRsCan533duEGw
5vAe61D-AAjGZBFA4Ep8mQ
9MLkFUNj7s1DnkGDv4DjyQ
w0ZJzBv7PVq2B_JHZswwbA
XX0yTQj6EK98lttAPZrjUg
9V2PyXye8zgPT9Kat5rN2w
TXU6j2y9Nem41SxxUUgA6A
7aZf5c1UNotq4MabBXMZLA
oCvxJBOs-Mbg-4oanPO7lQ
Y6Vw0FF8Qz9XD8fY4Z9Vfg
c4ZZnvkiELWstUP9i3z1nw
7tfXfmcGwbtYzUC9pWhc9w
QiPJy7hjfAMfa3RfBcx7Pw
nHFJU-_aHs4bprLhtN047A
H2RgkjPs3W-CxbyygCJJew
SnL1Fou0_JMlhULRky32Og
wjh07zG1xUKPQxduSGblSw
1qtS3V08sXD

JJU3d5aP1EErWc2PAZtzBA
fjOPc4qOLd7QLCeQfJ_F2A
i8e_c0PBfJvokjPNHhHEeQ
FVDxXrnzDsKMjklwjqB2SQ
s3Zs0mJLqxBbmpFm7OsMZQ
ngDI7XtTpP-E-Ttom3Gq1w
PasieMnGERmp41MnciVGZw
c-_7vBtzIfx_ZrE8eHBB2A
2XIvAOQ4SaZN5GB5qBiuhQ
oxABKl7Ol44cHxe7LfC-bA
n9LpCOXWJKHhwUAiwP62uQ
3MoNq4UCtJV2y1_aQ8-F9w
IHLWV3wACiH54VQuEfcEWQ
KzE_vf1xWUiFHSktVW2oFw
0CAyvTA4l5AzIIT5wiHUAg
1hp4WQLG8xVKaNUqPWqEYA
Ex1s434OH86X7Ue8z6Zblg
D2jB9_3_W2mTbSah9mx0ng
-ipcdpleKiq0citASTguEQ
Y9kXVlU4xPfSZvkZH6C_2Q
zr93wrNyXzc-HW4IcK4iRQ
o86gkXtslXkF-ZX63Ci-Iw
zZIhaE3yMCUCJpKsj0oOew
KB9ojpV8g5G1hHS6ec4eHg
NOdSt1Ku8s272TIT9TNzgA
4A0DRoZUsC27OuC6yi5rKw
JNuUI6PCRVKdlz_Bwv6rgA
HydP7LxrLmh_DMBsGgC1CQ
ycw145OEmoJGhTaFOwyk5A
4rnfwRvwAHSQfsihafmaXA
tOV1TLVcifCIxKcWljeesA
tbMIW8GSCBgpYOfKlaQXSg
po1Fj2KLsJEmDNdcJk40KA
V_2YX2rW_l5N3UwwifokCQ
lk02Yf1xJ-8Fg2b-2fTZow
vKKIO3Dc8m6i1C9UYwJzSQ
NxeEdKkvbZmuuBar_1NFrg
5S3wSG1Qm0tV07NACtzRCA
NxEFScwN858CmFo2ZtYQAA
_anMkOs2LhCPvH8T1Q9Cdw
3MHl5O3p5Hvvz4nmEHTp_g
gqs8RvcvId3TmZWBj1KKMg
2lpTbgwlVEYXgFBqUpKmrg
vwfDXpqtNO2

9ulYt6HD4-dJ6FXeP4pAwQ
eBj7zTaKvx_f80hdPE1sOg
L1UbHyyoYhgg49ihqEjOKA
QVMJ3o3ULCV3gENrvmk51A
o9Mnc5qhB83s30_fPCSb8w
1Kq_blM7ROYESz6i6FI_QQ
0txNpCq2YunZSL7jHIiGlA
_6hReDyVSr94W2WMO1LJLw
TffDDLGfej2dqD9LPEuArQ
lkM72Y21bjBqUGaW7iL7tQ
I62j-QfY4tO_gMKV-celEA
PMsqD6wHCfxdrZkGXIepvA
TzNye4Ff7QW0DoxXwO1LMw
8xCiibfgqy-8xpCWEaguqg
S5MYcMCSgqGJRLO8N5Uk3Q
9_CGhHMz8698M9-PkVf0CQ
pVmayyCiAscnNj3sAf08ow
Ojcf57dMPCW-WexHc6WS9w
vYKZHWBuBT1PUevggLAE9w
ovFnrmsT5rUpbAQE1r-V0Q
SpXBHrRfXJdocVkQa16zxg
skyDTwAT2-yM72Ce_JMFsw
evhhJ-3v9wIdVFGH_JsEww
79zzolCRbeM_1YjnssWBjw
NMEYiVsrV_S-snWCdDxFMw
onA8WmxUdcU5OdXwnLMHvA
0jANG8owjfQ66fPPB2eBgA
flyefPxFXUSD1f9u3kAumw
yUO7zPWGbumX42UzjbuBmw
4UmjnX46qbhV6-D5W89IOQ
WO1kqacT_65MYW8DTeydkQ
5oVIydJLDwm9chThPU6zIA
57yC5FzXbY59kOdVqR2FDw
G7Ull-QUaNXyPwtEErHZ8g
YRdqKN9G3F_xvqA-ANSC_Q
HVpwpXneaCWMeEBF7H8jpQ
2Euc8sFhQDvUEQgvd0XV9w
BQExW8hmH90ou7ol-91RMw
bffQssCZ-KpA_PPIu6Vysg
zB_qKuXiztrTsmLdag3AUQ
d7SRYwi0ukaxCTXL2vrlnA
YOZgLDLNLAe2nP1sw5inYw
7C1g2lil87sbKI3TDCMzxA
7e88qWZtdon

...

fzxvmA3Ygu3yrQQ6-cEpTw
c4zgJxmqz9nbBHF69EskSA
3hwX3hWFJaH1Ehd1eiJDfw
4oQSMGp3RweLCvGWuiZuCQ
GfoCqO0yYiXxTjkyDvSeSw
jAty2pgHVlp0sLyQo05r6Q
TxvTNCvCx8mn1E4DCojErQ
bVPTxKhjj4qyTF7_43nY-g
FdRHqEdH5QIFdWGQ9CcFSw
e4iYJVyoUAKlitSnxjgwbg
CEw4oD41d-B8GIHJAGLKyw
vcIEUwTx7S_6jaW0xlez_A
LuLb8a6Xf29f5NpLaPHNAg
gwo0iQlgQIKGXGTJYqWc7w
mPautiUsJGa4EsyY4ZCAqQ
O8ciwfMBzZ4FdBSXJ02fsQ
PFr2uIXNFKya1Fy46dW-zg
r4IirGbc1lWa17KI60oY9A
-KIdCJnkt5N8rnnmWR5MQg
grf5Gmn_GAh8ImHTi3HLPg
fuYRvZr6_XnhNW8PIQl6jg
BhB_WwI0TUdacBefXxhGgA
--ttCFj_csKJhxnaMRNuiw
vE5_WwToI4gaJj5tN-MhKA
AyrAB7r4rJAz0QrRtCScNQ
2-_UPP_gKEsx2F8BXZf-PA
ux1BtWYebgcNqwFvMBH_kw
tH9XWcFavOQ7BHch5YWE2A
CPRr6JgarnsJZFuvREtqnQ
jxbYiLM-HAAtuaT5O36-nA
PWBTdJNLnWcQcw9XD__WIA
mGLdVErt5PEpcvUJEk3CAg
jmTirQw-n4V_Z4g1-RE9iw
FyI7E0N9HzrgsgjWYJhLQA
tyZsR5D9gamqZ6uR5lA_yA
OTrOMhpSqz8wCaONMz8HEA
Y55vx2Bj9dREg_e6UdZ_cw
606lHID5NgVXKaODWnS1oA
96trsoEBDzmH-FDiVw4clg
D1Zz5Ea3kgsHUMDH0M-jNA
NPK7gCmgQE-13qOATboAQw
zkieW2F82--bbOIRhOCj0g
QRMpm9EpiY78YqWRIOjdKg
6x5rSZT_Ime

1SbEg-7CiMe8rDy8y9On9g
zoAj6LfU-2V60_P98s5nNw
OXCOUZ4c8xUu6o8PB0Mkqw
jcZXbQ96bznQyNIR0VxRtg
kOUVTxFC9usP6MyIRLZyCg
KJseeBAHS_FjHhUTtX9jFQ
vxX-Djz9mJke2uROm3ci-w
jVkhDTJXlTbfaGMfchfhPw
Y7OvwOdMgRiWARHM7M6H_A
KTgbFortwSnFSsQDhoXpTA
fjRWdBuZdkViPdXVVFLf4g
O_jKyEgP2uD7bD8Exl-law
kuQkVK8zYkY_jDW4D3F7Kw
jNTRcV8Xg3nbrkwovisrDg
zpCS2Gt8MOPiiIham-OBKw
tlKvIVn5KbUdyuh7_P12zg
1P7p44hzE3OIbYbuH5Ucaw
BVHK2GVfd9t5ViXIOFzzhw
UUCfOj7dq2WFwJGxJK3cQQ
u-Dst0PsuNvwzB_0o1JXBg
kUtlxm0SUJpHx8pzmesE9Q
eJKnymd0BywNPrJw1IuXVw
0PlZmy62LZmy5mA2BE6znw
hoiMOXY9hF3KKxVpF5dRsA
Ic1JQMZmghcf6fU7-7AYUw
HroRqAmZWXywHNUCRsaQeg
uWuaiOphdMd_9cKJpVgzhA
9Z_6rRy7Tl_C6HIgm7y6FA
pDGScweOylg1TQyaYfJqaw
tZEW8ESjrVmNGXvPnrR97w
d6_51PI94EC1C5sG9hi6HA
oT82GLKa-ntmiV00sd6PGA
E1eUQ9vJCMO3h9NJZfbhrg
YHeYZe4zPcEJ_Ph1hvfAdA
kn6KOuQXlRxbLhOAluNFpQ
bfIizwyT3-QXrntWbBpfKA
FTRKltZ9x7Q6oj-apyrrGw
xBqQzSE7XU_3cu24ygSsYw
LkXLvwiYIgnOtr5EORwu-A
uNRBw0CIY_eB5R4tdyCn7A
hm5Tgbj-IWQqe-ImqnBaNg
vZu12JAudoXBv3-i0JmSiQ
-fhq3ydpr22WWHCMwaw8oQ
uGIsWa5apwt

YPSpAMNFy2CJfELJWvHoKg
22ILFUhd4QWgo5mX2DBhhw
qmAM3dPGk04BOJYPxXDLyA
hfCWnw336ZBd8d_HzVwHwQ
BybnaYQUeCQfOyrYXnZOjg
oIl-Sh7ujuEHg0EEPKd3Qg
rYI_jy7zVVMSntuc2-C3rg
EMtSiPXlQtkhU0thhw2laQ
5iPdCrnosJg97DhNksl5uQ
dDHL1fouX6p7A2e-9PPLfw
DwfOyiI7x7ei2mHcoPwljQ
TcHDCrAp9NfFP2wGzQ3sXw
gx-jYW2qSsdPhHQtgIoWTA
3TWXRZeytVSW-NYxj7pt2g
YOUWcOrwggTkHtsQkyGdSw
p7j5VuH9L-5Rj7l7oT8-ww
05Gzb-XmOHibRxJOB0VOpA
6C2xFX5Q9mUmua0b4oSHvw
mbBhSRB_bpA0trAvCYzOuw
fP74vGq4DhnuT42LK_ojwg
qCTwVLF00fDJW3DHXBLsYQ
s8v3azqR8I68J7W2nMHh_w
JrT8i-pFwH55_hwP1haDzw
M3wSBWUoIARukqmhud1EgA
idDPf9jSyFoME81IB4EvNQ
FQq3yB79K40ya6dNjIqaBg
Gi5V4AIkSRXTJHSvGHB1Tw
MDBdzwiZr9ddieWeow7x7w
ZONtqsnEl47Zn2NunJYHIg
VawD7unTvrWLF14aYkt98g
vPr-KsosYalg8ZPr4CwXlQ
41Kbugm-qmIQTt7unkALYQ
AueEbb9kao4lt6CYt06hqw
vh1tPEaPioD78QmoqnWXpw
RhO--HyXPiV74--xF8eyUw
ciHtNE7zKzr25IX_L6pNSQ
u0G7IsCv4vZvLAwdN6i0sA
w-pXv6yfjkA0wi-yi3F8UQ
sym2kFcEcJ_tBDt85YoeGg
dErg5GT2TZctBRyx3pZDbg
24S-aiManGfjbb-fwwi2VQ
-JASfyJTDv4XImAxhj-ueQ
SeC2tDFHRGKwzsuZGYVBxA
hqJMHaqpyL6

VdwNUcgsLkdu5pEY7ovcwg
f7iqdZt45nYxdq2tshw5-w
wzKapkcKZRz209RI5LZyRg
10GxV6JOKPoR1w5KC2iZjA
8kL_Z36p2HtSe6-lh0naEQ
IpuLXGV2iSsQhejPIM3NQA
uls_TrWsOQqIKqHggq2rHA
JePAAhbOLBDI74VeePOBKw
TVKenh65WwJX3zEr36zKCA
vpwyL6NHm-pTWJ5IeJb9yw
V_VQ9BgbLEzL9mPLQQlc6g
QPsbqGvEy4xTwPBPoxd0Dg
HbWnJNtr8DITc4DkpvZKqg
iQOz4hIO6WDiu8fTy3iXpQ
qnuBmj3lL_S_QQOmTI9cfw
N-jRXGtQjEqQ0seqgqf8Ug
nCHs1mANWBdFIAYBKdUy4Q
RK2BzLXCIRLsIblJEK6R1w
AShjimfb4-Acstqtg9OYrg
hge6TQIwfwK3Xbq17GTO7w
JZXLgxJLKd9T-R6hrJhRZg
Z2GIrARCKaBy7Jr8Y-Sccw
SfLq0-xkvfTrntANX74apw
Giw0OL-ey7MqysdAVdkhfg
OkGSdziHWy6V3zhXNWU-Kg
TTZeAqyG5L5JzoX6aMhfHA
p7POIbP8XAGkaYa0W9SyOw
Nvr7_yjdolr1Hm54eyzTuw
fFjcz3U6YTXgOel9zOJ-8g
rKBqoXbY70awQP6p68e1_w
_1tgjKKo6tRoPXnBUOMqxQ
XIQZC49qv_75_3sKQApSsw
Vj2eN9PFTwkYByWRCknnMA
Nk2WUtjwUzCBWJavBEksJQ
U73q64yVTVTKM0RgF8pIag
RunrkT29nCOdQK7qS1Wqrg
d9TD1wi8Ndb-FAMrzwLidw
7maw_SNG0Q_co6-jPDoUnw
iQ1Jdz_4my9xjCeJIA59-Q
m_72eIu-gFJI8s17AJrsWQ
xFr7WKEyRRURjQmYXEk-LA
e-JcWplcnIx6lRmQoRF-Dw
rSjatGwSr86751cppmlBTA
Q2c0lgmYMk-

5eqm4pyz-egd255X7PB8Xg
35lDiS--Zx-8HLwv2EZBGQ
0d0i0FaJq1GIeW1rS2D-5w
10Jk5ilimXrfAq8JJ1gISg
e7207sqC-pSn6GIf31ikhQ
CEA3qbFhIHMhp1Ndpuunww
tEaW0zgsYihSTjysrpXaWg
XLc5WIzafCocqsycQhvEPw
QmVF3KJ07PsdroCFwOC_Ow
XORXa-bSuAgbk7_yXrP2cw
zLcarNPsB3Z2olCEjwbLuQ
lN4wmlhRagSZMF6YlOGNtg
W2W3tTQkid1Z5PXTnVQP6Q
Q4AWWdBWe_ojYX1a4ffXFQ
28uZ4-7Va3TkdU4hMmW4Qg
wvpeEJHhNpy7Bk_sKGvLcw
gZByUgIJsYIcZCGs--4Zhw
qiR7S83_7Du_XQyj_U0zWw
1fZuXWkUfdwT76oKGOhhUw
fZU9ZoMmuzO-b5071LvrWg
0bwQWqTDtGL1O32Kbkyd3Q
Ghelc8FIybfkzm96vzwcqQ
f13UZ4kKKRWZsbjmWnj2BA
Q-5sqRN6p4B9dgUQKA4hTw
-2AQs-KdwG5QCwGBSsDoCA
-JgxoIIxZf0q-PGD7P96Fg
GpawXT0eroxrkMqsv4ISKg
Cg_LgWgylKNkYLJNWv5ymw
_mx6QkDZ50ZgeNz81YWjrA
A_e5r8RK_rwrVPS2AE8ElA
sfUdeyZUl8gEWpCbnXTp2Q
3UPfd9cmG4OiP4j3LDnHFA
HTZFw4QDKhD8KRbnb5ZuJA
pqtBHj-aywRHfY8g9eCzqQ
REpKWYvpkE0O27AMf1iNIg
roZXPDSlvQcxwYah1Wa_0A
3HwoIoJcV7yDi3RV7T-oDQ
PMpHbI6pMmeV0R13dKFVzw
VUdydxqu1Dfsc4vubAGiZw
OKDm7TgreR14gs52t6Fb-Q
rLbZAC63Fo1bgBZGXqInlA
z3reQo0VWJtAg29xKp1wHA
5xg8O9rPksLss_I85tct-Q
KF-ajeGsyj6